# Build a Question/Answering system over SQL data

In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain langchain-community langchain-google-genai faiss-cpu langchain-groq langgraph

In [2]:
!apt-get install sqlite3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  sqlite3-doc
The following NEW packages will be installed:
  sqlite3
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 768 kB of archives.
After this operation, 1,873 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 sqlite3 amd64 3.37.2-2ubuntu0.3 [768 kB]
Fetched 768 kB in 1s (1,391 kB/s)
Selecting previously unselected package sqlite3.
(Reading database ... 123594 files and directories currently installed.)
Preparing to unpack .../sqlite3_3.37.2-2ubuntu0.3_amd64.deb ...
Unpacking sqlite3 (3.37.2-2ubuntu0.3) ...
Setting up sqlite3 (3.37.2-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...


In [3]:
from google.colab import userdata

from operator import itemgetter
import ast
import re

import sqlite3

from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_core.messages import SystemMessage
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_groq.chat_models import ChatGroq

In [4]:
!wget https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql

--2024-08-18 12:06:36--  https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611447 (597K) [text/plain]
Saving to: ‘Chinook_Sqlite.sql’

Chinook_Sqlite.sql  100%[===================>] 597.12K  --.-KB/s    in 0.04s   

2024-08-18 12:06:36 (16.0 MB/s) - ‘Chinook_Sqlite.sql’ saved [611447/611447]



In [5]:
sql_file_name = "/content/Chinook_Sqlite.sql"
db_name = "/content/Chinook.db"
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

with open(sql_file_name, 'r') as f:
    sql_script = f.read()

# Execute the script
cursor.executescript(sql_script)

# Commit changes
conn.commit()

# Close the connection
conn.close()

In [6]:
!sqlite3 Chinook.db "SELECT * FROM Artist LIMIT 10;"

1|AC/DC
2|Accept
3|Aerosmith
4|Alanis Morissette
5|Alice In Chains
6|Antônio Carlos Jobim
7|Apocalyptica
8|Audioslave
9|BackBeat
10|Billy Cobham


In [7]:
LANGCHAIN_API_KEY = userdata.get('LANGCHAIN_API_KEY')
LANGCHAIN_TRACING_V2 = userdata.get('LANGCHAIN_TRACING_V2')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
COHERE_API_KEY = userdata.get('COHERE_API_KEY')

In [8]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

# Convert question to SQL query

In [9]:
llm = ChatGroq(model="llama3-8b-8192", api_key=GROQ_API_KEY, stop_sequences=[])

In [10]:
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})
print(response)

Question: How many employees are there
SQLQuery: SELECT COUNT(*) FROM "Employee"


In [11]:
# Extract the SQL query from the response
sql_query = response.split("SQLQuery: ")[1]

# Execute the extracted SQL query
db.run(sql_query)

'[(8,)]'

In [12]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

# Execute SQL query

In [13]:
def remove_sql_query_prefix(text: str) -> str:
  """Removes the "SQLQuery: " prefix from the given text."""
  return text.split("SQLQuery: ")[1]

In [14]:
execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db) | remove_sql_query_prefix
chain = write_query | execute_query
chain.invoke({"question": "How many employees are there"})

'[(8,)]'

# Answer the question

In [16]:
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "How many employees are there"})

'According to the SQL result, [(8,)], there are **8** employees.'

# Agents

In [17]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

print(tools)

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f00b1bf5390>), InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f00b1bf5390>), ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7f00b1bf5390>), QuerySQLCheckerTool(description='Use this tool to double check if

In [18]:
SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [19]:
agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)

<ipython-input-19-f0adcce6690e>:1: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.3.0. Use 'state_modifier' parameter instead.
  agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)


In [20]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Which country's customers spent the most?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cdw2', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}, {'id': 'call_57p4', 'function': {'arguments': '{"table_names":"customer_table"}', 'name': 'sql_db_schema'}, 'type': 'function'}, {'id': 'call_qzg5', 'function': {'arguments': '{"query":"SELECT country, SUM(amount) AS total_spent FROM customer_table GROUP BY country ORDER BY total_spent DESC LIMIT 1"}', 'name': 'sql_db_query'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 181, 'prompt_tokens': 1633, 'total_tokens': 1814, 'completion_time': 0.150833333, 'prompt_time': 0.246343265, 'queue_time': None, 'total_time': 0.39717659800000005}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-085856df-bfa1-42f4-8e78-712c2f6d0574-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_cdw2'

# Dealing with high-cardinality columns

In [21]:
def query_as_list(db, query):
    res = db.run(query)
    # Flatten and Filter
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))

In [22]:
artists = query_as_list(db, "SELECT Name FROM Artist")
artists[:5]

['Corinne Bailey Rae',
 'The Black Crowes',
 'The Flaming Lips',
 'Marillion',
 'Barry Wordsworth & BBC Concert Orchestra']

In [23]:
albums = query_as_list(db, "SELECT Title FROM Album")
albums[:5]

['Use Your Illusion II',
 'Unplugged [Live]',
 'Beethoven Piano Sonatas: Moonlight & Pastorale',
 'Górecki: Symphony No.',
 'Black Sabbath']

In [24]:
vector_db = FAISS.from_texts(artists + albums, GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=GOOGLE_API_KEY
    ))
retriever = vector_db.as_retriever(search_kwargs={"k": 5})
description = """Use to look up values to filter on. Input is an approximate spelling of the proper noun, output is \
valid proper nouns. Use the noun most similar to the search."""
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

In [25]:
print(retriever_tool.invoke("Alice Chains"))

Alice In Chains

Carried to Dust (Bonus Track Version)

No More Tears (Remastered)

Unplugged [Live]

Pure Cult: The Best Of The Cult (For Rockers, Ravers, Lovers & Sinners) [UK]


In [26]:
system = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

You have access to the following tables: {table_names}

If you need to filter on a proper noun, you must ALWAYS first look up the filter value using the "search_proper_nouns" tool!
Do not try to guess at the proper name - use this function to find similar ones.""".format(
    table_names=db.get_usable_table_names()
)

system_message = SystemMessage(content=system)

tools.append(retriever_tool)

agent = create_react_agent(llm, tools, messages_modifier=system_message)

<ipython-input-26-88c1cb04fff6>:23: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.3.0. Use 'state_modifier' parameter instead.
  agent = create_react_agent(llm, tools, messages_modifier=system_message)


In [27]:
for s in agent.stream(
    {"messages": [HumanMessage(content="How many albums does alis in chain have?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_j856', 'function': {'arguments': '{"query":"SELECT COUNT(*) FROM Album WHERE ArtistName = \'Alice in Chains\'"}', 'name': 'sql_db_query'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 1826, 'total_tokens': 1905, 'completion_time': 0.065833333, 'prompt_time': 0.373423331, 'queue_time': None, 'total_time': 0.439256664}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_873a560973', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-894603b1-e661-4f02-9ff7-286c8b617a01-0', tool_calls=[{'name': 'sql_db_query', 'args': {'query': "SELECT COUNT(*) FROM Album WHERE ArtistName = 'Alice in Chains'"}, 'id': 'call_j856', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1826, 'output_tokens': 79, 'total_tokens': 1905})]}}
----
{'tools': {'messages': [ToolMessage(content="Error: (sqlite3.OperationalError) no such column: ArtistName\n